# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment,Workspace
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_failure_prediction'

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()
path = "https://raw.githubusercontent.com/Sara-cos/Heart_Faliure_PredictionModel/main/starter_file/heart_failure_clinical_records_dataset.csv"
dataset = Dataset.Tabular.from_delimited_files(path)

Workspace name: quick-starts-ws-142842
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-142842


In [7]:
cpu_compute_name = "cpucompute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name= cpu_compute_name)
    print('Using existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_compute_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Using existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = cpu_cluster,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",   
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpucompute with default configuration
Running on remote compute: cpucompute


Experiment,Id,Type,Status,Details Page,Docs Page
heart_failure_prediction,AutoML_2585cd63-a50b-46ad-bea9-b2a5134d816f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart_failure_prediction,AutoML_2585cd63-a50b-46ad-bea9-b2a5134d816f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_2585cd63-a50b-46ad-bea9-b2a5134d816f',
 'target': 'cpucompute',
 'status': 'Completed',
 'startTimeUtc': '2021-04-16T11:38:33.993525Z',
 'endTimeUtc': '2021-04-16T12:09:15.652857Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpucompute',
  'AMLSettingsJsonString': '{"path":null,"name":"heart_failure_prediction","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-142842","workspace_name":"quick-starts-ws-142842","region":"southcentralus","compute_target":"cpucompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, best_model = remote_run.get_output()
print("Best run ID:"+ best_run.id)
print(best_run)
print(best_model)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

Best run ID:AutoML_2585cd63-a50b-46ad-bea9-b2a5134d816f_44
Run(Experiment: heart_failure_prediction,
Id: AutoML_2585cd63-a50b-46ad-bea9-b2a5134d816f_44,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.05263157894736842,
                                                                   

In [13]:
#TODO: Save the best model
model = best_run.register_model(model_name='heart-failure-prediction-automl-model',model_path='outputs/model.pkl', tags={'Method':'AutoML'}, properties={'accuracy': best_run_metrics['accuracy']})
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-142842', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-142842'), name=heart-failure-prediction-automl-model, id=heart-failure-prediction-automl-model:1, version=1, tags={'Method': 'AutoML'}, properties={'accuracy': '0.8728735632183909'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.webservice import Aciwebservice, Webservice
from azureml.core.model import Model, Inferenceconfig
from azureml.core import Environment
from aure_ml.core.conda_dependencies import CondaDependencies

env = Environment.from_conda_specification(name='automlv2', file_path='conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

service_name = 'heartfailurepredictionautomlv2'

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1.8, memory_gb = 4, auth_enabled = True, enable_app_insights = True)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
print(service.state)

service.wait_for_deployment(show_output = True)
url = service.scoring_uri
print(url)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

data = {"data":
        [
          {
            "age": 36,
            "anaemia": 0,
            "creatinine_phosphokinase": 200,
            "diabetes": 0,
            "ejection_fraction": 30,
            "high_blood_pressure": 0,
            "platelets": 120000,
            "serum_creatinine": 1.1,
            "serum_sodium": 135,
            "sex": 1,
            "smoking": 0,
            "time": 7
          },
          {
            "age": 39,
            "anaemia": 0,
            "creatinine_phosphokinase": 300,
            "diabetes": 0,
            "ejection_fraction": 50,
            "high_blood_pressure": 0,
            "platelets": 200000,
            "serum_creatinine": 0.9,
            "serum_sodium": 250,
            "sex": 0,
            "smoking": 0,
            "time": 1
          }
      ]
    }

input_payload = json.dumps(data)

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#service.delete()